In [ ]:
from all_support_functions import *

In [ ]:
class Parser:
    def __init__(self, link, out_file):
        self.link = link
        self.out_file = out_file

    
    def build_urls(self, listings_per_page=20, pages_per_location=1):
        """Builds links for all search pages for a given location"""
        url_list = []
        for i in range(pages_per_location):
            offset = listings_per_page * i
            url_pagination = self.link + f'&items_offset={offset}'
            url_list.append(url_pagination)
            self.url_list = url_list


    def process_search_pages(self):
        """Extract features from all search pages"""
        features_list = []
        for page in self.url_list:
            listings = extract_listings(page)
            for listing in listings:
                features = extract_listing_features(listing, RULES_SEARCH_PAGE)
                features['sp_url'] = page
                features_list.append(features)
        
        self.base_features_list = features_list

        

    def process_detail_pages(self):
        print("in the details")
        """Runs detail pages processing in parallel"""
        pool = multiprocessing.Pool()
        pool = multiprocessing.Pool(processes=4)
        result = pool.map(scrape_detail_page, self.base_features_list)
        pool.close()
#         pool.join()
#         result=[]
#         #print(self.base_features_list)
#         for i in self.base_features_list:
#             result.append(scrape_detail_page(i))
        self.all_features_list = result


    def save(self, feature_set='all'):
        if feature_set == 'basic':
            pd.DataFrame(self.base_features_list).to_csv(self.out_file, index=False)
        elif feature_set == 'all':
            pd.DataFrame(self.all_features_list).to_csv(self.out_file, index=False)
        else:
            pass
            
        
    def parse(self):
        self.build_urls()
        self.process_search_pages()
        self.process_detail_pages()
        self.save('all')

In [ ]:
%%time
AIRBNB_LINK = 'https://www.airbnb.com/s/Capitola--California--United-States/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_lengths%5B%5D=one_week&price_filter_input_type=0&price_filter_num_nights=5&query=Capitola%2C%20CA&place_id=ChIJ066GeHQVjoAR58t1aK7zMNw&date_picker_type=calendar&source=structured_search_input_header&search_type=autocomplete_click'
if __name__ == "__main__":
    new_parser = Parser(AIRBNB_LINK, './test.csv')
    t0 = time.time()
    new_parser.parse()
    print(time.time() - t0)